# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

Answer: This is a classification problem, because we are putting the students into distinct groups: ones that might need help, and those that don't. Regression is used more for continuous data.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
# Subtract one since 'passed' is a target and not a feature
n_features = len(student_data.columns) - 1

# TODO: Calculate passing students 
n_passed = np.sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
n_failed = n_students - n_passed

# TODO: Calculate graduation rate
grad_rate = float(n_passed) / float(n_students) * 100 

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [5]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
# I set the training size in the function below

# Set the number of testing points
# I set the test size in the function below

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, stratify = y_all, train_size=300, test_size=95, random_state=23)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

Answer: I picked Decision Trees, Support Vector Machines (SVM) and AdaBoost (one of the Ensemble methods).

Decision Trees:
1) Real world application - Could be used for disease diagnosis - based on certain symptoms, could branch out into various possible diagnoses. For instance, a runny nose could be indicative of a cold or a flu, so you could continue down branches that possibly lead to those diagnoses, but a runny nose is probably not indicative of a skin infection. Got this idea from http://www.cbcb.umd.edu/~salzberg/docs/murthy_thesis/survey/node32.html, which actually lists a ton of great real world uses. 
2) Strengths - Fairly easy to understand and does not take a lot of data prepping. Decision trees can also use both categorical or numerical data (such as splitting runny nose above). They also have fairly strong performance on large datasets.
3) Weaknesses - Can overfit due to over-complicated trees. In other words, it might look great on the training data but not on the test set. This actually ends up happening below, as the training sets come out perfect while it is one of the lower test scores.
4) Candidacy - Although we actually ended up processing the data above to be numerical, the data fits to decision trees in a fairly straightforward way, and I can think in my head exactly how it might be setting itself up behind the scenes. So part of it is being able to know exactly how it is going to look at the data - for instance, I bet the tree is going to split on whether someone has failed a class before.

SVM:
1) Real world application - I looked a little at this website: http://www.clopinet.com/SVM.applications.html, but other than the ones listed, I also think that using SVMs in stock price analysis would be useful. SVMs can handle multiple features fairly easily, so I think putting in multiple things such as P/E ratio, revenue growth, analyst opinions (average positivity), and even odd ones like social media buzz (on a range of 1-100 perhaps) could arrive at a decent model of where the stock price is going to move.
2) Strengths - Good at attempting to minimize error while maximizing the margin (i.e. finding the classification separation where most of the points are the most separate with least errors), and also good with multiple dimensions.
3) Weaknesses - Speed and size - can be a little slow depending on the data set, as well as eating up a decent amount of memory for larger tasks.
4) Candidacy - One of the main items I remembered from the lecture videos was that SVMs did well with multiple dimensions. Due to the decent amount of features in the data set, I think this part of SVMs (good with multi-dimensionality) should do well with this data (and it does!).

AdaBoost:
1) Real world application - Viola-Jones Face Detection (as mentioned here: http://mccormickml.com/2013/12/13/adaboost-tutorial/). Face detection is really big right now, especially with social media companies - Facebook uses it to detect who is in photos posted, and Snapchat uses it to map various filters to your face.
2) Strengths - By going back and weighting on the errors it made to try to get closer to the one it previously got wrong (and placing less weight on what it got right), this algorithm specifically targets what it is getting wrong and tries to improve. Then, it averages itself over a number of rounds. In theory, averaging multiple rounds of this is going to be very accurate.
3) Weaknesses - A problem can arrive in the strength above in that it can also cause overfitting when it weights toward the ones it was previously getting wrong. Also, needs an underlying algorithm that can use weighting. 
4) Candidacy - From the lecture videos, they seemed to put a high emphasis on how effective boosting can be, which is why I chose it. I actually figured that boosting with SVMs would work really well, while boosting with Decision Trees was probably less likely to work well, but the default is to use Decision Trees (done below). Although I ended up going with SVM at the end of this project due to speed, I played around with a tuned Adaboost over multiple algorithms, and SVM was the one the tuner chose, fairly in line with my expectation.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [29]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.svm import SVC
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the three models
clf_A = SVC(random_state = 23)
clf_B = tree.DecisionTreeClassifier(random_state = 23)
clf_C = AdaBoostClassifier(random_state = 23)

# TODO: Set up the training set sizes
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(X_all, y_all, stratify = y_all, train_size=100, random_state=23)

X_train_200, X_test_200, y_train_200, y_test_200 = train_test_split(X_all, y_all, stratify = y_all, train_size=200, random_state=23)

X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(X_all, y_all, stratify = y_all, train_size=300, random_state=23)

# TODO: Execute the 'train_predict' function for each classifier and each training set size
# train_predict(clf, X_train, y_train, X_test, y_test)
train_predict(clf_A, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_A, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_A, X_train_300, y_train_300, X_test_300, y_test_300)
train_predict(clf_B, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_B, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_B, X_train_300, y_train_300, X_test_300, y_test_300)
train_predict(clf_C, X_train_100, y_train_100, X_test_100, y_test_100)
train_predict(clf_C, X_train_200, y_train_200, X_test_200, y_test_200)
train_predict(clf_C, X_train_300, y_train_300, X_test_300, y_test_300)


Training a SVC using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0010 seconds.
F1 score for training set: 0.8323.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8065.
Training a SVC using a training set size of 200. . .
Trained model in 0.0040 seconds
Made predictions in 0.0030 seconds.
F1 score for training set: 0.8758.
Made predictions in 0.0020 seconds.
F1 score for test set: 0.8129.
Training a SVC using a training set size of 300. . .
Trained model in 0.0080 seconds
Made predictions in 0.0070 seconds.
F1 score for training set: 0.8584.
Made predictions in 0.0030 seconds.
F1 score for test set: 0.8212.
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0020 seconds
Made predictions in 0.0000 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0000 seconds.
F1 score for test set: 0.7475.
Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

Classifer 1 - SVM

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |        0.0020           |         0.0020         |      0.8323      |     0.8065      |
| 200               |        0.0040           |         0.0020         |      0.8758      |     0.8129      |
| 300               |        0.0080           |         0.0030         |      0.8584      |     0.8212      |

Classifier 2 - Decision Tree 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.0020          |         0.0000         |      1.0000      |     0.7475      |
| 200               |         0.0020          |         0.0010         |      1.0000      |     0.7536      |
| 300               |         0.0030          |         0.0000         |      1.0000      |     0.7132      |

Classifier 3 - Adaboost 

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |         0.1450          |         0.0060         |      0.9343      |     0.7389      |
| 200               |         0.1480          |         0.0050         |      0.8834      |     0.7900      |
| 300               |         0.1570          |         0.0050         |      0.8519      |     0.8148      |

## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

Answer: I would choose the Support Vector Machine (SVM). First of all, it performed extremely well regardless of training test size (~6% better at only 100 training points), and continues to be the best up to the 300 training point size. It has higher training and prediction time than the tree, but the tree just doesn't perform near as well due to constant overfitting (evidenced by the decision tree perfecting the training set each time, while never hitting 80% on the test set). Also, it is faster than Adaboost (especially in training) while still ending up slightly better even at the 300 training set size.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

Answer: My chosen model, SVM, works by trying to maximize the "margin" between classes (individuals at-risk versus low risk) while also striving for the lowest amount of incorrect classes. The margin is based on where the boundaries of the classes get set (which the algorithm moves to maximize this margin). These boundaries are based on the algorithm looking at a subset of points around a given point for similarity; these points in the subset are the "support vectors". This helps to cut down on the processing time in predictions. These support vectors are weighted when we train the algorithm to be more or less important, to help further refine the accuracy. 

It does well with multiple features, such as the included after-school activities, age, previous failed classes, etc. Essentially, it can combine a whole lot of categories of information into two distinct classes very accurately! It includes what is called the "kernel trick" - basically, it helps measure the similarity of two different points in higher dimensions, so that while in a 2D space they might not seem to be separable, adding an extra dimension makes them easier to classify.

The below image helps show this "higher dimension" action (from http://www.eric-kim.net/eric-kim-net/posts/1/kernel_trick.html):

http://www.eric-kim.net/eric-kim-net/posts/1/imgs/data_2d_to_3d.png

In the image, there is not linear separation on the left image, but in a 3D space (the right side), you can more easily see where a line would fall (across the Z-axis).

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [30]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import fbeta_score, make_scorer

# TODO: Create the parameters list you wish to tune
parameters = {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'], 'C' : [0.1,1.0,10], 'gamma' : ['auto',0.1,1.0,10]}

# TODO: Initialize the classifier
clf = SVC(random_state = 23)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(fbeta_score, beta = 1, pos_label = 'yes')

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid=parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

Made predictions in 0.0030 seconds.
Tuned model has a training F1 score of 0.8279.
Made predictions in 0.0010 seconds.
Tuned model has a testing F1 score of 0.8194.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

Answer: The final model has a F1 score for training of 0.8279 and testing of 0.8194. This is interesting as it actually came out fairly comparable to the default settings on the untuned model - likely due to SVM being a pretty good fit for use on this dataset. 

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.